In [1]:
import torch
from torch import nn

from torch.optim import AdamW

from torchcrf import CRF

from transformers import BertModel, BertConfig

In [22]:
bert_config = BertConfig.from_pretrained("bert-base-cased", num_labels=9)
model = BertModel(bert_config)

rnn_dim = 128

In [2]:
bilstm = nn.LSTM(bert_config.hidden_size, 128, num_layers=1, bidirectional=True, batch_first=True)

optimizer = AdamW(model.parameters(), lr=2e-5)

NameError: name 'bert_config' is not defined

In [28]:
crf = nn.Sequential(
            nn.Linear(128 * 2, bert_config.num_labels),
            CRF(bert_config.num_labels),
        )

In [29]:
class CustomModel(nn.Module):
    def __init__(self, bert_config, args):
        super(CustomModel, self).__init__()
        self.bert = BertModel(bert_config)
        self.birnn = nn.LSTM(bert_config.hidden_size, args.rnn_dim, num_layers=1, bidirectional=True)
        self.linear = nn.Linear(args.rnn_dim * 2),
        self.crf = CRF(bert_config.num_labels)

        self.dropout = nn.Dropout(0.1)

    def forward(self, **batch):
        bert_embedding = self.bert(**batch)[0]
        bilstm_embedding = self.birnn(bert_embedding)
        bilstm_embedding = self.dropout(bilstm_embedding)

        low_dimension_embedding = self.linear(bilstm_embedding)
        loss = -1 * self.crf(low_dimension_embedding, batch["labels"], attention_mask=batch["attention_mask"])

        return loss

    def predict(self, **batch):
        with torch.no_grad():
            bert_embedding = self.bert(**batch)[0]
            bilstm_embedding = self.birnn(bert_embedding)
            bilstm_embedding = self.dropout(bilstm_embedding)

            low_dimension_embedding = self.linear(bilstm_embedding)
            predictions = self.crf.decode(low_dimension_embedding, attention_mask=batch["attention_mask"])

            return predictions